In [27]:
import pandas as pd

# Load the csv into pandas with appropriate encoding to avoid UnicodeDecodeError
# Skip the first row as it is useless
df = pd.read_csv('./stadat-ara0040-1.2.1.2-hu.csv', encoding="iso-8859-2", sep=";")

# Display the full dataframe output as a scrollable element
import IPython
IPython.display.display(df)





,"1.2.1.2. A fogyasztóiár-index fogyasztási főcsoportok szerint, és a nyugdíjas fogyasztóiár-index, havonta",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,
0,Év,Időszak,Élelmiszerek,"Szeszes italok, dohányáruk",Ruházkodási cikkek,Tartós fogyasztási cikkek,"Háztartási energia, fűtés","Egyéb cikkek, üzemanyagok",Szolgáltatások,Összesen,Nyugdíjas fogyasztóiár-index
1,"Az előző év azonos időszaka = 100,0%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021.,január,"103,9","109,9","97,9","103,1","100,3","99,9","101,7","102,7","102,9"
3,NaN,február,"103,4","109,9","98,4","103,8","100,3","102,7","101,7","103,1","103,2"
4,NaN,március,"102,7","110,3","98,2","103,6","100,3","107,1","101,3","103,7","103,4"
...,...,...,...,...,...,...,...,...,...,...,...
235,NaN,januáraugusztus,"106,2","106,1","101,9","102,0","104,6","101,0","106,8","104,7","104,8"
236,NaN,januárszeptember,"106,0","106,2","101,9","102,0","105,3","100,9","106,7","104,6","104,7"
237,NaN,januároktóber,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,NaN,januárnovember,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:

# Now add back the very first 2021 január row for each df to start with 2021 január.
columns = [
    "Év",
    "Időszak",
    "Élelmiszerek",
    "Szeszes italok, dohányáruk",
    "Ruházkodási cikkek",
    "Tartós fogyasztási cikkek",
    "Háztartási energia, fűtés",
    "Egyéb cikkek, üzemanyagok",
    "Szolgáltatások",
    "Összesen",
    "Nyugdíjas fogyasztóiár-index"
]

def fix_dash(s):
    # Replace problematic unicode dash (0x96, '') with normal '-'
    if isinstance(s, str):
        return s.replace('\x96', '-')
    return s

with open("./stadat-ara0040-1.2.1.2-hu.csv", encoding="iso-8859-2") as f:
    lines = f.readlines()

# Find the first full data row (2021 január)
jan_row = None
for i, l in enumerate(lines):
    if l.strip().startswith("2021.;janu"):
        jan_row = l.strip().split(';')
        # Pad/cut if necessary
        jan_row = jan_row[:11] + [""] * (11 - len(jan_row))
        break

header_lines_enc = [
    "Az el\u0151z\u0151 \u00e9v azonos id\u0151szaka = 100,0%",
    "El\u0151z\u0151 h\u00f3nap = 100,0%",
    "El\u0151z\u0151 \u00e9v december = 100,0 %",
    "\u00c9vkezdett\u0151l kumul\u00e1lt, el\u0151z\u0151 \u00e9v azonos id\u0151szaka = 100,0%"
]

# Find where each block starts
marker_indices = []
for i, line in enumerate(lines):
    clean = line.strip().replace('\ufeff', '')
    for mark in header_lines_enc:
        if clean.startswith(mark):
            marker_indices.append(i)
marker_indices.append(len(lines))

dfs = []
for idx in range(4):
    start = marker_indices[idx]
    end = marker_indices[idx+1]
    block_lines = lines[start:end]
    if len(block_lines) < 2:
        dfs.append(pd.DataFrame(columns=columns))
        continue
    data_lines = block_lines[2:]
    # Remove blanks/short
    data_lines = [dl for dl in data_lines if dl.strip()]
    parsed_rows = []
    for dl in data_lines:
        cells = dl.strip().split(";")
        cells = cells[:11] + [""] * (11 - len(cells))
        if any(cell.strip() for cell in cells):
            parsed_rows.append(cells)
    # Insert the január row at the beginning if it's not already there
    if jan_row and (not parsed_rows or parsed_rows[0][0] != "2021." or (parsed_rows[0][1].lower() != "január" and parsed_rows[0][1].lower() != "janu\u00e1r") ):
        parsed_rows.insert(0, jan_row)
    # For the last block, fix encoding for each cell
    if idx == 3:
        parsed_rows = [[fix_dash(cell) for cell in row] for row in parsed_rows]
    df_blk = pd.DataFrame(parsed_rows, columns=columns)
    dfs.append(df_blk)

df_azonos_idoszak, df_honap, df_december, df_kumulalt = dfs

import IPython
print("Az előző év azonos időszaka blokk:")
IPython.display.display(df_azonos_idoszak.head(5))
print("Előző hónap blokk:")
IPython.display.display(df_honap.head(5))
print("Előző év december blokk:")
IPython.display.display(df_december.head(5))
print("Évkezdettől kumulált blokk:")
IPython.display.display(df_kumulalt.head(5))








Az előző év azonos időszaka blokk:


,Év,Időszak,Élelmiszerek,"Szeszes italok, dohányáruk",Ruházkodási cikkek,Tartós fogyasztási cikkek,"Háztartási energia, fűtés","Egyéb cikkek, üzemanyagok",Szolgáltatások,Összesen,Nyugdíjas fogyasztóiár-index
0,2021.,január,"103,9","109,9","97,9","103,1","100,3","99,9","101,7","102,7","102,9"
1,,február,"103,4","109,9","98,4","103,8","100,3","102,7","101,7","103,1","103,2"
2,,március,"102,7","110,3","98,2","103,6","100,3","107,1","101,3","103,7","103,4"
3,,április,"102,4","112,2","99,8","103,4","100,4","113,9","102,0","105,1","104,3"
4,,május,"102,6","112,2","101,0","103,5","100,4","113,4","102,1","105,1","104,4"


Előző hónap blokk:


,Év,Időszak,Élelmiszerek,"Szeszes italok, dohányáruk",Ruházkodási cikkek,Tartós fogyasztási cikkek,"Háztartási energia, fűtés","Egyéb cikkek, üzemanyagok",Szolgáltatások,Összesen,Nyugdíjas fogyasztóiár-index
0,2021.,január,"103,9","109,9","97,9","103,1","100,3","99,9","101,7","102,7","102,9"
1,,február,"101,0","100,3","99,8","100,4","100,0","101,3","100,5","100,7","100,7"
2,,március,"100,4","101,3","100,6","99,9","100,0","102,5","100,1","100,7","100,6"
3,,április,"101,1","102,4","103,0","100,3","100,0","100,0","100,5","100,8","100,7"
4,,május,"100,8","99,9","100,9","100,7","100,0","100,8","100,2","100,5","100,5"


Előző év december blokk:


,Év,Időszak,Élelmiszerek,"Szeszes italok, dohányáruk",Ruházkodási cikkek,Tartós fogyasztási cikkek,"Háztartási energia, fűtés","Egyéb cikkek, üzemanyagok",Szolgáltatások,Összesen,Nyugdíjas fogyasztóiár-index
0,2021.,január,"103,9","109,9","97,9","103,1","100,3","99,9","101,7","102,7","102,9"
1,,február,"102,1","102,6","97,0","101,1","100,1","102,9","101,0","101,6","101,7"
2,,március,"102,4","103,9","97,6","101,0","100,1","105,5","101,1","102,4","102,2"
3,,április,"103,6","106,4","100,6","101,3","100,1","105,5","101,6","103,2","102,9"
4,,május,"104,4","106,3","101,4","102,0","100,1","106,3","101,8","103,7","103,5"


Évkezdettől kumulált blokk:


,Év,Időszak,Élelmiszerek,"Szeszes italok, dohányáruk",Ruházkodási cikkek,Tartós fogyasztási cikkek,"Háztartási energia, fűtés","Egyéb cikkek, üzemanyagok",Szolgáltatások,Összesen,Nyugdíjas fogyasztóiár-index
0,2021.,január,"103,9","109,9","97,9","103,1","100,3","99,9","101,7","102,7","102,9"
1,,január-március,"103,4","110,0","98,2","103,5","100,3","103,2","101,6","103,2","103,1"
2,,január-április,"103,1","110,6","98,6","103,5","100,3","105,9","101,7","103,6","103,4"
3,,január-május,"103,0","110,9","99,0","103,5","100,3","107,4","101,8","103,9","103,6"
4,,január-június,"103,0","111,1","99,4","103,5","100,3","107,9","102,1","104,2","103,8"


In [ ]:
inflation_category_colours = {
    "Élelmiszerek": "#1f77b4",  # blue
    "Szeszes italok, dohányáruk": "#ff7f0e",  # orange
    "Ruházkodási cikkek": "#2ca02c",  # green
    "Tartós fogyasztási cikkek": "#d62728",  # red
    "Lakbér, háztartási energia": "#9467bd",  # purple
    "Egyéb cikkek, üzemanyag": "#8c564b",  # brown
    "Szolgáltatások": "#e377c2",  # pink
    "Élelmiszer, szeszes ital, dohányáruk": "#7f7f7f",  # gray
    "Háztartási energia": "#bcbd22",  # olive
    "Közlekedés": "#17becf",  # cyan
    "Nyugdíjas fogyasztóiár-index": "#aec7e8",  # light blue
}
